PUNTO 3: CROSS VALIDATION

1. Importar librerías

In [1]:
#importamos todas las liberías necesarias. 
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler, MinMaxScaler
from sklearn.metrics import balanced_accuracy_score, accuracy_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
import pickle


2. Carga de datos

In [2]:
# Cargar los datos (reemplaza con tu archivo específico)
available_data_path = f"attrition_availabledata_03.csv"
df = pd.read_csv(available_data_path)

3. Eliminación de datos irrelevantes

    Estas columnas son identificadores "EmployeeID" y valores constantes

In [3]:
# Eliminar columnas irrelevantes
drop_columns = ["EmployeeID", "Over18", "StandardHours", "EmployeeCount"]
df = df.drop(columns=drop_columns, errors='ignore')


4. Convertir todas las variables a tipo numérico y separar según características y variable objetivo (Attrition)

In [4]:
# Convertir variables categóricas a numéricas usando Label Encoding
categorical_columns = df.select_dtypes(include=['object']).columns
label_encoders = {}

for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le  # Guardar para futuras conversiones si es necesario

# Separar en características (X) y variable objetivo (y)
X = df.drop(columns=["Attrition"])
y = df["Attrition"]

5. Preprocesamiento de datos: detección de outliers y escalado

In [5]:
df = X

# Seleccionar columnas numéricas
numerical_cols = df.select_dtypes(include=
[np.number]).columns.tolist()

# Función para detectar outliers usando regla IQR
def porcentaje_outliers(col):
    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1
    # Definir límites
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outlier_count = ((col < lower_bound) | (col > upper_bound)).sum()
    return outlier_count / len(col)

# Umbrales para la recomendación
umbral_skew = 1.0       # asimetría mayor a 1 (o menor a -1) se considera alta
umbral_out = 0.05       # Si más del 5% de las instancias son outliers

print("Recomendación de transformación para cada columna numérica:")
print("-" * 70)

robust_columns = []
standard_columns = []

for col in numerical_cols:
    serie = df[col]
    skew = serie.skew()
    outlier_pct = porcentaje_outliers(serie)
    desc = serie.describe()
    
    print(f"\nColumna: {col}")
    print(f"  Media: {desc['mean']:.3f}, Std: {desc['std']:.3f}")
    print(f"  Mín: {desc['min']}, Máx: {desc['max']:.3f}")
    print(f"  Asimetría (skew): {skew:.3f}")
    print(f"  Porcentaje de outliers (IQR): {outlier_pct*100:.2f}%")
    
    # Lógica de recomendación
    if abs(skew) > umbral_skew or outlier_pct > umbral_out:
        robust_columns.append(col)
        print("  -> Sugerencia: La distribución es sesgada y/o presenta outliers; se recomienda normalización o escalado robusto.")
    else:
        standard_columns.append(col)
        print("  -> Sugerencia: Distribución aproximadamente normal; se recomienda estandarización con StandardScaler.")

Recomendación de transformación para cada columna numérica:
----------------------------------------------------------------------

Columna: hrs
  Media: 7.326, Std: 1.334
  Mín: 5.4168797411869445, Máx: 10.937
  Asimetría (skew): 0.859
  Porcentaje de outliers (IQR): 2.62%
  -> Sugerencia: Distribución aproximadamente normal; se recomienda estandarización con StandardScaler.

Columna: absences
  Media: 12.702, Std: 5.518
  Mín: 1.0, Máx: 24.000
  Asimetría (skew): 0.015
  Porcentaje de outliers (IQR): 0.00%
  -> Sugerencia: Distribución aproximadamente normal; se recomienda estandarización con StandardScaler.

Columna: JobInvolvement
  Media: 2.740, Std: 0.718
  Mín: 1.0, Máx: 4.000
  Asimetría (skew): -0.516
  Porcentaje de outliers (IQR): 0.00%
  -> Sugerencia: Distribución aproximadamente normal; se recomienda estandarización con StandardScaler.

Columna: PerformanceRating
  Media: 3.161, Std: 0.367
  Mín: 3.0, Máx: 4.000
  Asimetría (skew): 1.847
  Porcentaje de outliers (IQR): 16

5.1. Intanciación y aplicación de escaladores

In [6]:
robust_scaler = RobustScaler()
standard_scaler = StandardScaler()

# Aplicar las transformaciones de forma separada
X_transformed = X.copy()

if robust_columns:
    X_transformed[robust_columns] = robust_scaler.fit_transform(X[robust_columns])
if standard_columns:
    X_transformed[standard_columns] = standard_scaler.fit_transform(X[standard_columns])

6. Division de datos y configuración de Cross-Validation

    Se usa el parametro stratify para mantener la proporción de clases en Train y Test. Debido al desbalanceo de clases que se vio en el EDA.

In [7]:
# Dividir en Train (2/3) y Test (1/3)
X_train, X_test, y_train, y_test = train_test_split(X_transformed, y, test_size=1/3, random_state=3, stratify=y)

# Guardar el scaler para futuros usos
with open("robust_scaler.pkl", "wb") as f:
    pickle.dump(robust_scaler, f)
with open("standard_scaler.pkl", "wb") as f:
    pickle.dump(standard_scaler, f)

# Evaluación INNER: K-Fold Cross Validation (k=5)
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=3)

print("Datos preparados: Train (2/3), Test (1/3), y Cross-Validation lista.")


Datos preparados: Train (2/3), Test (1/3), y Cross-Validation lista.


7. Evaluación outer: Estimación de rendimiento con un modelo base

In [8]:
# Entrenar modelo base en Train
model = DecisionTreeClassifier(random_state=3)
model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=3)

In [9]:
# Predecir en el conjunto de Test (Outer Evaluation)
y_pred = model.predict(X_test)

In [10]:
# Cálculo de métricas
balanced_acc = balanced_accuracy_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

In [11]:
# Matriz de confusión para obtener TPR y TNR
conf_matrix = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = conf_matrix.ravel()

In [12]:
# True Positive Rate (TPR) y True Negative Rate (TNR)
TPR = tp / (tp + fn)
TNR = tn / (tn + fp)

# 📌 Mostrar resultados
print(f"Balanced Accuracy: {balanced_acc:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"True Positive Rate (TPR): {TPR:.4f}")
print(f"True Negative Rate (TNR): {TNR:.4f}")
print(f"Matriz de Confusión:\n{conf_matrix}")

Balanced Accuracy: 0.8580
Accuracy: 0.9204
True Positive Rate (TPR): 0.7658
True Negative Rate (TNR): 0.9501
Matriz de Confusión:
[[781  41]
 [ 37 121]]


5. HPO:  REVISAR

In [13]:
# # Definir el espacio de hiperparámetros a optimizar
# param_grid = {
#     'max_depth': [3, 5, 7, None],
#     'min_samples_split': [2, 5, 10]
# }

# # Usar el mismo kfold definido para la evaluación inner
# grid_search = GridSearchCV(
#     estimator=DecisionTreeClassifier(random_state=3),
#     param_grid=param_grid,
#     scoring='balanced_accuracy',
#     cv=kfold,  # Evaluación inner con validación cruzada estratificada (k=5)
#     n_jobs=-1
# )

# # Ajustar en el conjunto de entrenamiento
# grid_search.fit(X_train, y_train)

# # Imprimir los mejores hiperparámetros y la métrica de inner evaluation
# print("Mejores hiperparámetros:", grid_search.best_params_)
# print("Puntuación inner (Balanced Accuracy):", grid_search.best_score_)

--------------------------------------------------------------------------------------

PUNTO 4: MÉTODOS BÁSICOS (KNN y Árboles de decisión)

Escalar e imputar usando KNN como referencia

In [14]:
scalers = {
    "StandardScaler": StandardScaler(),
    "MinMaxScaler": MinMaxScaler(),
    "RobustScaler": RobustScaler()
}

imputers = {
    "Mean": SimpleImputer(strategy="mean"),
    "Median": SimpleImputer(strategy="median")
}

Evaluando con KNN (k=5) para decidir el mejor método

In [15]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=3)
best_score = 0
best_scaler = None
best_imputer = None

for scaler_name, scaler in scalers.items():
    for imputer_name, imputer in imputers.items():
        X_train_imputed = imputer.fit_transform(X_train)
        X_train_scaled = scaler.fit_transform(X_train_imputed)
        
        knn = KNeighborsClassifier(n_neighbors=5)
        score = np.mean(cross_val_score(knn, X_train_scaled, y_train, cv=kfold, scoring="balanced_accuracy"))
        
        print(f"Scaler: {scaler_name}, Imputer: {imputer_name}, Score: {score:.4f}")
        
        if score > best_score:
            best_score = score
            best_scaler = scaler
            best_imputer = imputer

Scaler: StandardScaler, Imputer: Mean, Score: 0.5922
Scaler: StandardScaler, Imputer: Median, Score: 0.5928
Scaler: MinMaxScaler, Imputer: Mean, Score: 0.5922
Scaler: MinMaxScaler, Imputer: Median, Score: 0.5903
Scaler: RobustScaler, Imputer: Mean, Score: 0.6000
Scaler: RobustScaler, Imputer: Median, Score: 0.5987


Aplicar el mejor preprocesamiento encontrado

In [16]:
X_train_imputed = best_imputer.fit_transform(X_train)
X_test_imputed = best_imputer.transform(X_test)
X_train_scaled = best_scaler.fit_transform(X_train_imputed)
X_test_scaled = best_scaler.transform(X_test_imputed)

print(f"\nMejor método de escalado: {best_scaler}, Mejor método de imputación: {best_imputer}")


Mejor método de escalado: RobustScaler(), Mejor método de imputación: SimpleImputer()


Entrenando y evaluando modelos base (KNN y Árboles de Decisión)

In [17]:
# definimos modelos base
knn = KNeighborsClassifier(n_neighbors=5)
tree = DecisionTreeClassifier(random_state=3)

# evaluamos con Cross-Validation
knn_score = np.mean(cross_val_score(knn, X_train_scaled, y_train, cv=kfold, scoring="balanced_accuracy"))
tree_score = np.mean(cross_val_score(tree, X_train_scaled, y_train, cv=kfold, scoring="balanced_accuracy"))

print(f"\nBalanced Accuracy KNN: {knn_score:.4f}")
print(f"Balanced Accuracy Árbol de Decisión: {tree_score:.4f}")


Balanced Accuracy KNN: 0.6000
Balanced Accuracy Árbol de Decisión: 0.8180


Optimización de hiperparámetros (HPO) con GridSearchCV

In [18]:
# definimos grids de hiperparámetros
knn_params = {"n_neighbors": [3, 5, 7, 9, 11]}
tree_params = {"max_depth": [3, 5, 10, None], "min_samples_split": [2, 5, 10]}

# optimizamos con GridSearchCV
knn_grid = GridSearchCV(KNeighborsClassifier(), knn_params, cv=kfold, scoring="balanced_accuracy", n_jobs=-1)
tree_grid = GridSearchCV(DecisionTreeClassifier(random_state=3), tree_params, cv=kfold, scoring="balanced_accuracy", n_jobs=-1)

# entrenamos GridSearchCV
knn_grid.fit(X_train_scaled, y_train)
tree_grid.fit(X_train_scaled, y_train)

# obtenemos mejores parámetros y scores
best_knn = knn_grid.best_estimator_
best_tree = tree_grid.best_estimator_

print(f"\nMejor KNN: {knn_grid.best_params_}, Balanced Accuracy: {knn_grid.best_score_:.4f}")
print(f"Mejor Árbol: {tree_grid.best_params_}, Balanced Accuracy: {tree_grid.best_score_:.4f}")


Mejor KNN: {'n_neighbors': 3}, Balanced Accuracy: 0.6589
Mejor Árbol: {'max_depth': None, 'min_samples_split': 2}, Balanced Accuracy: 0.8180


Evaluación final en el conjunto de Test (Outer)

In [20]:
# predecimos y evaluamos
y_pred_knn = best_knn.predict(X_test_scaled)
y_pred_tree = best_tree.predict(X_test_scaled)

# Métricas para cada modelo
def evaluar_modelo(y_test, y_pred, model_name):
    balanced_acc = balanced_accuracy_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = conf_matrix.ravel()
    TPR = tp / (tp + fn)
    TNR = tn / (tn + fp)
    
    print(f"\n{model_name} - Evaluación en Test Set")
    print(f"Balanced Accuracy: {balanced_acc:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"True Positive Rate (TPR): {TPR:.4f}")
    print(f"True Negative Rate (TNR): {TNR:.4f}")
    print(f"Matriz de Confusión:\n{conf_matrix}")

evaluar_modelo(y_test, y_pred_knn, "KNN")
evaluar_modelo(y_test, y_pred_tree, "Árbol de Decisión")


KNN - Evaluación en Test Set
Balanced Accuracy: 0.6814
Accuracy: 0.8643
True Positive Rate (TPR): 0.4114
True Negative Rate (TNR): 0.9513
Matriz de Confusión:
[[782  40]
 [ 93  65]]

Árbol de Decisión - Evaluación en Test Set
Balanced Accuracy: 0.8559
Accuracy: 0.9255
True Positive Rate (TPR): 0.7532
True Negative Rate (TNR): 0.9586
Matriz de Confusión:
[[788  34]
 [ 39 119]]
